# Script para extraer información del Directorio de Proveedores y Contratistas Sancionados.

Directorio de Licitantes, Proveedores y Contratistas sancionados con el impedimento para presentar propuestas o celebrar contratos con las dependencias, entidades de la Administración Pública Federal y de los Gobiernos de los Estados.

Ultima fecha de actualización:03/Marzo/2025

In [22]:
import requests
import json
import pandas as pd

url = 'https://servicios-estrategico.funcionpublica.gob.mx/particularesSancionadosPro'

# Modificamos el payload eliminando el filtro de numero_expediente y rfc para extraer todosblos expedientes,
#si se desea un expediente en particular se agrega siguiendo la sintaxis ---->  filtros: {numero_expediente: "expediente que se busca", rfc: "rfc que se busca"}
payload = {
    "operationName": "getExpediente",
    "variables": {
        "filtros": {}  # Aquí vaciamos el filtro para que devuelva todos los expedientes
    },
    "query": """
    query getExpediente($filtros: FiltrosInput!) {
        results(filtros: $filtros) {
            numero_expediente
            numero_procedimiento_adm
            numexpediente_corto
            numprocedimiento_corto
            rfc
            nombre_razon_social
            institucion_dependencia {
                nombre
            }
            fecha_notificacion
            multa {
                monto
                monto_cf
            }
            plazo {
                fecha_inicial
                fecha_final
                plazo_inha
            }
            observaciones
            objeto_social
            leyes_infringidas
            causa_motivo_hechos
            autoridad_sancionadora
            fecha_dof
            responsable {
                nombres_resp
                primer_apellido_resp
                segundo_apellido_resp
                cargo_resp
                telefono_resp
                email_resp
            }
            telefono
        }
    }
    """
}

response = requests.post(url, json=payload)
data = response.json()

# Imprimir los resultados en formato JSON
#with open('respuesta_expedientes.json', 'w') as f:
  #  json.dump(data, f, indent=4)


Se desea obtener la información en formato estructurado

In [39]:
# Extraer los resultados de la respuesta
results = data.get('data', {}).get('results', [])

# Si la clave 'results' existe, la convertimos a un DataFrame
if results:
    # Crear un DataFrame a partir de la lista de resultados
    df = pd.json_normalize(results, sep='_')

    # Mostrar el DataFrame
    print(df.head(3))  # Mostrar las primeras filas para ver cómo luce
else:
    print("No se encontraron expedientes.")

  numero_expediente numero_procedimiento_adm numexpediente_corto  \
0    090850001/2024        09085PA-0007/2024           0001/2024   
1    000070021/2022        00007PA-0021/2022           0021/2022   
2    006410117/2023        00641PA-0045/2023           0117/2023   

  numprocedimiento_corto            rfc  \
0           PA-0007/2024  LOLD860221ND2   
1           PA-0021/2022   CPU150813UE7   
2           PA-0045/2023   SSS160921JC6   

                          nombre_razon_social fecha_notificacion  \
0                    DANIEL GASPAR LOPEZ LANZ         2024-12-17   
1          COMERCIALIZADORA PUAN S.A. DE C.V.         2024-07-16   
2  SYS SUMINISTROS Y SOLUCIONES, S.A. DE C.V.         2025-02-13   

                                       observaciones objeto_social  \
0  LA SANCIÓN SE ENCUENTRA SUBSISTENTE, NO HA INT...          None   
1  SE INHABILITA POR EL TERMINO DE 7 MESES Y UNA ...          None   
2                       YA SE NOTIFICO LA RESOLUCION          None   



Se guarda en un archivo excel

In [49]:
df.to_excel('prov_sancionados_contratistas.xlsx', index=False, engine='openpyxl')